In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Bidirectional,SpatialDropout1D
from keras.models import Model
from keras.models import Sequential

In [2]:
data = pd.read_csv('../preprocess_data.csv')
data.drop(['task_1','text'], axis=1, inplace=True)
data.head()

,_id,task_2,task_3,task_4,text_clean
0,Hindi_image_1817.jpg,Neutral,Vulgar,Abusive,Ba8@ DaNn G@rainiD IR T३ PDBB WRHE W PRD BCEN ...
1,Hindi_image_7.jpg,Negative,Vulgar,Abusive,"Nari nari mat kar pagle, Nari he nark ka dwar...."
2,Hindi_image_1.jpg,Positive,Non Vulgar,Abusive,Kitni push ops maarsakte ho dafly? 5 aur agar ...
3,Hindi_image_32.jpg,Negative,Vulgar,Abusive,अब इसमें मेरी कहां गलती है बताओ.. तरबूज़ वाली क...
4,Hindi_image_1714.jpg,Negative,Non Vulgar,Abusive,"""KUDI MENU KEHNDl... 'MENU JUTI LA DE SONIYE....."


In [5]:
test_data = pd.read_csv('../preprocess_test_data.csv')
test_data.drop(['task_1','Unnamed: 0','text'], axis=1, inplace=True)
test_data.head()

,_id,task_2,task_3,task_4,text_clean
0,Hindi_image_410.jpg,Positive,Vulgar,Non-abusive,"Sign You are Bancho a] _ ~""11|7 have best ffen..."
1,Hindi_image_114.jpg,Negative,Vulgar,Abusive,एक महिला घोडे़ के लिंग लिया| घोड़ा उत्साहित हो...
2,Hindi_image_101.jpg,Neutral,Non Vulgar,Non-abusive,एक टीचर ने एक लड़के को पेपर में नक़ल करते पकड लि...
3,Hindi_image_1747.jpg,Negative,Vulgar,Abusive,show me Sckht Launda Kisslay Jha CTrollerlzabu...
4,Hindi_image_19.jpg,Negative,Non Vulgar,Abusive,पति सुहागरात में पत्नी की निप्पल चूसते हुए बोल...


In [4]:
sentences = data['text_clean'].astype(str)
tokenizer = Tokenizer(num_words = 1500,split=' ')
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)

In [5]:
test_sentences = test_data['text_clean'].astype(str)
test_sequence = tokenizer.texts_to_sequences(test_sentences)

In [6]:
max_seq_len = 2500

index_of_words = tokenizer.word_index
print("No of unique words : ",len(index_of_words))

X = pad_sequences(sequence , maxlen = max_seq_len )
Y = data['task_2']

print(X)

No of unique words :  8859
[[   0    0    0 ... 1173 1174  571]
 [   0    0    0 ...   19   15 1176]
 [   0    0    0 ...    4  773   17]
 ...
 [   0    0    0 ...   36  377   30]
 [   0    0    0 ...   27  122  333]
 [   0    0    0 ...  118  739   89]]


In [7]:
test_X = pad_sequences(test_sequence , maxlen = max_seq_len )
test_Y = test_data['task_2']

print(test_X)

[[   0    0    0 ...   46  963    9]
 [   0    0    0 ...   45   35   26]
 [   0    0    0 ...  545  310    2]
 ...
 [   0    0    0 ...   87  143  333]
 [   0    0    0 ...    0    0  318]
 [   0    0    0 ...    0    0 1297]]


In [8]:
embed_dim = 256
vocabSize = len(index_of_words)
lstm_out = 64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 0)
Y_true = Y_test
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values

In [10]:
test_Y_true = test_Y
test_Y = pd.get_dummies(test_Y).values
print("test_Y:",test_Y)

test_Y: [[False False  True]
 [ True False False]
 [False  True False]
 ...
 [False False  True]
 [False  True False]
 [False  True False]]


# MODEL 1

In [11]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length = 2500))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         2267904   
                                                                 
 spatial_dropout1d (Spatial  (None, 2500, 256)         0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 2350275 (8.97 MB)
Trainable params: 2350275 (8.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("hasoc_a2.h5", monitor='val_loss', verbose=1, save_best_only=True,
save_weights_only=False, mode='auto')

In [13]:
print(Y_true)

1018    Positive
1020    Negative
870     Positive
184     Negative
686     Negative
          ...   
55      Negative
744     Positive
785      Neutral
946     Negative
295     Positive
Name: task_2, Length: 172, dtype: object


In [14]:
print(Y_test)
classes = ['Negative','Neutral','Positive']

[[False False  True]
 [ True False False]
 [False False  True]
 [ True False False]
 [ True False False]
 [False  True False]
 [False  True False]
 [ True False False]
 [False False  True]
 [False  True False]
 [ True False False]
 [ True False False]
 [ True False False]
 [False False  True]
 [False False  True]
 [ True False False]
 [False  True False]
 [ True False False]
 [ True False False]
 [ True False False]
 [False False  True]
 [ True False False]
 [False False  True]
 [False  True False]
 [False  True False]
 [ True False False]
 [ True False False]
 [False False  True]
 [False False  True]
 [ True False False]
 [False False  True]
 [ True False False]
 [False  True False]
 [False False  True]
 [ True False False]
 [False  True False]
 [False  True False]
 [ True False False]
 [False  True False]
 [False False  True]
 [False  True False]
 [ True False False]
 [False False  True]
 [ True False False]
 [ True False False]
 [ True False False]
 [False False  True]
 [ True False

In [15]:
model.fit(X_train, Y_train, batch_size = 8, epochs = 10 , validation_data = (X_test, Y_test), callbacks = [checkpoint])

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.6188 - accuracy: 0.4696
Epoch 1: val_loss improved from inf to 0.62955, saving model to hasoc_a2.h5
122/122 [==============================] - 235s 2s/step - loss: 0.6188 - accuracy: 0.4696 - val_loss: 0.6295 - val_accuracy: 0.4012
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - ETA: 0s - loss: 0.5648 - accuracy: 0.5635
Epoch 2: val_loss improved from 0.62955 to 0.60418, saving model to hasoc_a2.h5
122/122 [==============================] - 249s 2s/step - loss: 0.5648 - accuracy: 0.5635 - val_loss: 0.6042 - val_accuracy: 0.4767
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4347 - accuracy: 0.7069
Epoch 3: val_loss did not improve from 0.60418
122/122 [==============================] - 249s 2s/step - loss: 0.4347 - accuracy: 0.7069 - val_loss: 0.7060 - val_accuracy: 0.4186
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.2855 - accuracy: 0.8215
Epoch 4: val_loss did not improve from 0.60418
122/122 [==============================] - 246s 2s/step - loss: 0.2855 - accuracy: 0.8215 - val_loss: 0.8076 - val_accuracy: 0.4651
Epoch 5/10
122/122 [==============================] - ETA: 0s - loss: 0.1991 - accuracy: 0.8906
Epoch 5: val_loss did not improve from 0.60418
122/122 [=

In [16]:
model.load_weights('hasoc_a2.h5')
model.evaluate(X_test,Y_test)

6/6 [==============================] - 4s 620ms/step - loss: 0.6042 - accuracy: 0.4767


[0.6041780710220337, 0.4767441749572754]

In [17]:
Y_pred = model.predict(test_X)

25/25 [==============================] - 17s 665ms/step


In [18]:
print("Y_pred:",Y_pred)

[[0.03771292 0.07741701 0.88487005]
 [0.19440939 0.18823677 0.6173538 ]
 [0.61268383 0.146672   0.24064414]
 ...
 [0.37182772 0.15705739 0.47111484]
 [0.65076727 0.16120294 0.1880298 ]
 [0.7870871  0.11707791 0.09583499]]


In [22]:
pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))
print(pred_class)

pred_class_condition = pd.get_dummies(pred_class).values
print(pred_class_condition)

[2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [24]:
print(classification_report(test_Y, pred_class_condition))

              precision    recall  f1-score   support

           0       0.10      0.76      0.18        76
           1       1.00      0.00      0.01       648
           2       0.09      0.42      0.15        45

   micro avg       0.10      0.10      0.10       769
   macro avg       0.40      0.40      0.11       769
weighted avg       0.86      0.10      0.03       769
 samples avg       0.10      0.10      0.10       769



In [25]:
pred_actual = []
for i in pred_class:
    if i == 0:
        pred_actual.append('Negative')
    elif i == 1 :
        pred_actual.append('Neutral')
    else:
        pred_actual.append('Positive')

In [26]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_a2.csv',index=False)
test_data.head()

,_id,label
0,Hindi_image_410.jpg,Positive
1,Hindi_image_114.jpg,Positive
2,Hindi_image_101.jpg,Negative
3,Hindi_image_1747.jpg,Positive
4,Hindi_image_19.jpg,Positive


# MODEL 2

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Embedding层，增加防止过拟合的Dropout
model.add(Embedding(input_dim=vocabSize, output_dim=embed_dim, input_length=2500))
model.add(Dropout(0.3))

# LSTM层，增加recurrent_dropout 和 output dropout
model.add(LSTM(units=lstm_out, dropout=0.3, recurrent_dropout=0.3, return_sequences=False))

# Batch Normalization增强泛化
model.add(BatchNormalization())

# 全连接层，Softmax输出3分类，建议用categorical_crossentropy
model.add(Dense(3, activation='softmax'))

# 编译
optimizer = Adam(learning_rate=0.001)  # 学习率也可调整
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 256)         2267904   
                                                                 
 dropout (Dropout)           (None, 2500, 256)         0         
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 2350531 (8.97 MB)
Trainable params: 2350403 (8.97 MB)
Non-trainable params: 128 (512.00 Byte)
______________

In [12]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("hasoc_b2.h5", monitor='val_loss', verbose=1, save_best_only=True,
save_weights_only=False, mode='auto')

In [13]:
model.fit(X_train, Y_train, batch_size = 8, epochs = 10, validation_data = (X_test, Y_test), callbacks = [checkpoint])

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.1616 - accuracy: 0.3891
Epoch 1: val_loss improved from inf to 1.09600, saving model to hasoc_b2.h5
122/122 [==============================] - 333s 3s/step - loss: 1.1616 - accuracy: 0.3891 - val_loss: 1.0960 - val_accuracy: 0.3837
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - ETA: 0s - loss: 0.8200 - accuracy: 0.6460
Epoch 2: val_loss improved from 1.09600 to 1.08187, saving model to hasoc_b2.h5
122/122 [==============================] - 335s 3s/step - loss: 0.8200 - accuracy: 0.6460 - val_loss: 1.0819 - val_accuracy: 0.3837
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.6100 - accuracy: 0.7317
Epoch 3: val_loss improved from 1.08187 to 1.06746, saving model to hasoc_b2.h5
122/122 [==============================] - 338s 3s/step - loss: 0.6100 - accuracy: 0.7317 - val_loss: 1.0675 - val_accuracy: 0.4419
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.4266 - accuracy: 0.8318
Epoch 4: val_loss did not improve from 1.06746
122/122 [==============================] - 337s 3s/step - loss: 0.4266 - accuracy: 0.8318 - val_loss: 1.2577 - val_accuracy: 0.4419
Epoch 5/10
122/122 [==============================] - ETA: 0s - loss: 0.4071 - accuracy: 0.8452
Epoch 5: val_loss did no

In [14]:
model.load_weights('hasoc_b2.h5')
model.evaluate(X_test,Y_test)

6/6 [==============================] - 4s 649ms/step - loss: 1.0675 - accuracy: 0.4419


[1.067458987236023, 0.44186046719551086]

In [15]:
Y_pred = model.predict(test_X)

25/25 [==============================] - 17s 682ms/step


In [16]:
pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))
print(pred_class)

pred_class_condition = pd.get_dummies(pred_class).values
print(pred_class_condition)

[2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 2, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 2, 2, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 1, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 2, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 

In [17]:
print(classification_report(test_Y, pred_class_condition))

              precision    recall  f1-score   support

           0       0.10      0.76      0.18        76
           1       0.88      0.12      0.21       648
           2       0.05      0.13      0.07        45

   micro avg       0.19      0.19      0.19       769
   macro avg       0.34      0.34      0.16       769
weighted avg       0.75      0.19      0.20       769
 samples avg       0.19      0.19      0.19       769



In [18]:
pred_actual = []
for i in pred_class:
    if i == 0:
        pred_actual.append('Negative')
    elif i == 1 :
        pred_actual.append('Neutral')
    else:
        pred_actual.append('Positive')

In [19]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_b2.csv',index=False)
test_data.head()

,_id,label
0,Hindi_image_410.jpg,Positive
1,Hindi_image_114.jpg,Positive
2,Hindi_image_101.jpg,Negative
3,Hindi_image_1747.jpg,Negative
4,Hindi_image_19.jpg,Positive


# MODEL 3

In [11]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "hasoc_c2.h5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False, 
    mode='auto')

In [12]:
model = Sequential()
model.add(Embedding(vocabSize, embed_dim, input_length=2500))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight
# 假设Y_train已独热编码
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1, 2]), y=np.argmax(Y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

model.fit(X_train, Y_train, epochs=10, batch_size=8, class_weight=class_weight_dict, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0951 - accuracy: 0.3633
Epoch 1: val_loss improved from inf to 1.07616, saving model to hasoc_c2.h5
122/122 [==============================] - 332s 3s/step - loss: 1.0951 - accuracy: 0.3633 - val_loss: 1.0762 - val_accuracy: 0.4593
Epoch 2/10


C:\Users\8888\Anaconda3\envs\pythonProject11\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - ETA: 0s - loss: 0.9417 - accuracy: 0.6058
Epoch 2: val_loss did not improve from 1.07616
122/122 [==============================] - 332s 3s/step - loss: 0.9417 - accuracy: 0.6058 - val_loss: 1.0903 - val_accuracy: 0.4826
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.6107 - accuracy: 0.7441
Epoch 3: val_loss did not improve from 1.07616
122/122 [==============================] - 332s 3s/step - loss: 0.6107 - accuracy: 0.7441 - val_loss: 1.2450 - val_accuracy: 0.4302
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.3498 - accuracy: 0.8535
Epoch 4: val_loss did not improve from 1.07616
122/122 [==============================] - 331s 3s/step - loss: 0.3498 - accuracy: 0.8535 - val_loss: 1.6500 - val_accuracy: 0.4884
Epoch 5/10
122/122 [==============================] - ETA: 0s - loss: 0.2279 - accuracy: 0.9082
Epoch 5: val_loss did not improve from 1.07616
122/122 [==============================] - 

In [13]:
model.load_weights('hasoc_c2.h5')
model.evaluate(X_test,Y_test)

6/6 [==============================] - 4s 651ms/step - loss: 1.0762 - accuracy: 0.4593


[1.0761574506759644, 0.4593023359775543]

In [14]:
Y_pred = model.predict(test_X)

25/25 [==============================] - 17s 668ms/step


In [15]:
pred_class = []
for i in Y_pred:
    pred_class.append(np.argmax(i))
print(pred_class)

pred_class_condition = pd.get_dummies(pred_class).values
print(pred_class_condition)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 2, 0, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 2, 0, 2, 1, 0, 0, 2, 2, 1, 0, 1, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 1, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 1, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 1, 0, 1, 0, 2, 0, 0, 2, 0, 2, 2, 1, 0, 2, 1, 2, 2, 0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 2, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 1, 0, 0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 1, 2, 2, 0, 2, 1, 1, 0, 2, 1, 0, 2, 0, 2, 0, 0, 1, 2, 

In [16]:
print(classification_report(test_Y, pred_class_condition))

              precision    recall  f1-score   support

           0       0.10      0.36      0.16        76
           1       0.93      0.17      0.29       648
           2       0.08      0.64      0.14        45

   micro avg       0.22      0.22      0.22       769
   macro avg       0.37      0.39      0.19       769
weighted avg       0.80      0.22      0.27       769
 samples avg       0.22      0.22      0.22       769



In [17]:
pred_actual = []
for i in pred_class:
    if i == 0:
        pred_actual.append('Negative')
    elif i == 1 :
        pred_actual.append('Neutral')
    else:
        pred_actual.append('Positive')

In [18]:
test_data = test_data[["_id"]]
test_data["label"] = pred_actual
test_data.to_csv('dl_lstm_c2.csv',index=False)
test_data.head()

,_id,label
0,Hindi_image_410.jpg,Positive
1,Hindi_image_114.jpg,Positive
2,Hindi_image_101.jpg,Positive
3,Hindi_image_1747.jpg,Positive
4,Hindi_image_19.jpg,Positive
